Recurrent Neural networks
=====

### RNN  

<img src ="imgs/rnn.png" width="20%">

A recurrent neural network (RNN) is a class of artificial neural network where connections between units form a directed cycle. This creates an internal state of the network which allows it to exhibit dynamic temporal behavior.

In [1]:
import keras
output_dim = 100

Using TensorFlow backend.


In [2]:
keras.layers.recurrent.SimpleRNN(output_dim, 
                                 init='glorot_uniform', inner_init='orthogonal', activation='tanh', 
                                 W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                                 dropout_W=0.0, dropout_U=0.0)

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(100, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", dropout=0.0, recurrent_regularizer=None, activation="tanh", recurrent_dropout=0.0, bias_regularizer=None, kernel_regularizer=None)`


#### Backprop Through time  

Contrary to feed-forward neural networks, the RNN is characterized by the ability of encoding longer past information, thus very suitable for sequential models. The BPTT extends the ordinary BP algorithm to suit the recurrent neural
architecture.

<img src ="imgs/rnn2.png" width="45%">

In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing import image
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import imdb
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from keras.layers.core import Activation, RepeatVector
from keras.callbacks import EarlyStopping, ModelCheckpoint

# import tensorflow as tf

# tf.python.control_flow_ops = tf

#### IMDB sentiment classification task

This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. 

IMDB provided a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. 

There is additional unlabeled data for use as well. Raw text and already processed bag of words formats are provided. 

http://ai.stanford.edu/~amaas/data/sentiment/

#### Data Preparation - IMDB

In [27]:
max_features = 20000
maxlen = 100  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print("Loading data...")
# (X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features, test_split=0.2)
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Example:')
print(X_train[:1])

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

max_epoch = 10

Loading data...


/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/datasets/imdb.py:44: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


25000 train sequences
25000 test sequences
Example:
[ list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19

In [28]:
X_test[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     1,   591,   202,    14,
          31,     6,   717,    10,    10, 18142, 10698,     5,     4,
         360,     7,     4,   177,  5760,   394,   354,     4,   123,
           9,  1035,  1035,  1035,    10,    10,    13,    92,   124,
          89,   488,  7944,   100,    28,  1668,    14,    31,    23,
          27,  7479,    29,   220,   468,     8,   124,    14,   286,
         170,     8,   157,    46,     5,    27,   239,    16,   179,
       15387,    38,    32,    25,  7944,   451,   202,    14,     6,   717], dtype=int32)

In [29]:
X_train.shape

(25000, 100)

#### Model building 

In [30]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(SimpleRNN(128))  
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(X_test, y_test))

acc , loss = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...


/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 15s 594us/step

Test accuracy: 0.518018109779
Test loss: 0.74916


### LSTM  

A LSTM network is an artificial neural network that contains LSTM blocks instead of, or in addition to, regular network units. A LSTM block may be described as a "smart" network unit that can remember a value for an arbitrary length of time. 

Unlike traditional RNNs, an Long short-term memory network is well-suited to learn from experience to classify, process and predict time series when there are very long time lags of unknown size between important events.

<img src ="imgs/gru.png" width="60%">

In [8]:
keras.layers.recurrent.LSTM(output_dim, init='glorot_uniform', inner_init='orthogonal', 
                            forget_bias_init='one', activation='tanh', 
                            inner_activation='hard_sigmoid', 
                            W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                            dropout_W=0.0, dropout_U=0.0)

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, recurrent_dropout=0.0, kernel_regularizer=None, unit_forget_bias=True, activation="tanh", recurrent_initializer="orthogonal", kernel_initializer="glorot_uniform", dropout=0.0, bias_regularizer=None, recurrent_activation="hard_sigmoid", recurrent_regularizer=None)`


### GRU  

Gated recurrent units are a gating mechanism in recurrent neural networks. 

Much similar to the LSTMs, they have fewer parameters than LSTM, as they lack an output gate.

In [9]:
keras.layers.recurrent.GRU(output_dim, init='glorot_uniform', inner_init='orthogonal', 
                           activation='tanh', inner_activation='hard_sigmoid', 
                           W_regularizer=None, U_regularizer=None, b_regularizer=None, 
                           dropout_W=0.0, dropout_U=0.0)

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `GRU` call to the Keras 2 API: `GRU(100, recurrent_dropout=0.0, kernel_regularizer=None, recurrent_initializer="orthogonal", kernel_initializer="glorot_uniform", dropout=0.0, activation="tanh", recurrent_activation="hard_sigmoid", bias_regularizer=None, recurrent_regularizer=None)`


### Your Turn! - Hands on Rnn

In [10]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
model.add(SimpleRNN(128))  
#model.add(GRU(128))  
#model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))

loss , acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...


/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 99s 4ms/step - loss: 0.6901 - acc: 0.5593 - val_loss: 0.5484 - val_acc: 0.7257
Epoch 2/10
25000/25000 [==============================] - 99s 4ms/step - loss: 0.5001 - acc: 0.7698 - val_loss: 0.4544 - val_acc: 0.7967
Epoch 3/10
25000/25000 [==============================] - 99s 4ms/step - loss: 0.3779 - acc: 0.8444 - val_loss: 0.4368 - val_acc: 0.8174
Epoch 4/10
25000/25000 [==============================] - 99s 4ms/step - loss: 0.4799 - acc: 0.7736 - val_loss: 0.5347 - val_acc: 0.7623
Epoch 5/10
25000/25000 [==============================] - 99s 4ms/step - loss: 0.4837 - acc: 0.7744 - val_loss: 0.5025 - val_acc: 0.7602
Epoch 6/10
25000/25000 [==============================] - 102s 4ms/step - loss: 0.4415 - acc: 0.8048 - val_loss: 0.5491 - val_acc: 0.7690
Epoch 7/10
25000/25000 [==============================] - 103s 4ms/step - loss: 0.4417 - acc: 0.8036 - val_loss: 0.5575 - val_a

In [11]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
# model.add(SimpleRNN(128))  
model.add(GRU(128))  
# model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
loss , acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...


/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.4571 - acc: 0.7752 - val_loss: 0.3409 - val_acc: 0.8530
Epoch 2/10
25000/25000 [==============================] - 236s 9ms/step - loss: 0.2370 - acc: 0.9080 - val_loss: 0.3407 - val_acc: 0.8528
Epoch 3/10
25000/25000 [==============================] - 227s 9ms/step - loss: 0.1341 - acc: 0.9525 - val_loss: 0.4974 - val_acc: 0.8367
Epoch 4/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0695 - acc: 0.9757 - val_loss: 0.5114 - val_acc: 0.8390
Epoch 5/10
25000/25000 [==============================] - 234s 9ms/step - loss: 0.0349 - acc: 0.9895 - val_loss: 0.7263 - val_acc: 0.8381
Epoch 6/10
25000/25000 [==============================] - 227s 9ms/step - loss: 0.0252 - acc: 0.9918 - val_loss: 0.7254 - val_acc: 0.8378
Epoch 7/10
25000/25000 [==============================] - 226s 9ms/step - loss: 0.0210 - acc: 0.9937 - val_loss: 0.8580 - 

In [12]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))

# Play with those! try and get better results!
# model.add(SimpleRNN(128))  
# model.add(GRU(128))  
model.add(LSTM(128))  

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=["accuracy"])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, 
          nb_epoch=max_epoch, validation_data=(X_test, y_test))
loss , acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print()
print('Test accuracy:', acc)
print('Test loss:', loss)

Build model...
Train...


/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 278s 11ms/step - loss: 0.4262 - acc: 0.8031 - val_loss: 0.3457 - val_acc: 0.8472
Epoch 2/10
25000/25000 [==============================] - 272s 11ms/step - loss: 0.2443 - acc: 0.9043 - val_loss: 0.3556 - val_acc: 0.8462
Epoch 3/10
25000/25000 [==============================] - 271s 11ms/step - loss: 0.1531 - acc: 0.9449 - val_loss: 0.5806 - val_acc: 0.8261
Epoch 4/10
25000/25000 [==============================] - 306s 12ms/step - loss: 0.1080 - acc: 0.9616 - val_loss: 0.5681 - val_acc: 0.8336
Epoch 5/10
25000/25000 [==============================] - 328s 13ms/step - loss: 0.0766 - acc: 0.9738 - val_loss: 0.5304 - val_acc: 0.8276
Epoch 6/10
25000/25000 [==============================] - 345s 14ms/step - loss: 0.0604 - acc: 0.9791 - val_loss: 0.5783 - val_acc: 0.8257
Epoch 7/10
25000/25000 [==============================] - 275s 11ms/step - loss: 0.0469 - acc: 0.9844 - val_loss: 0.

In [13]:
model.evaluate?